# Build Pipeline

Speeds up build and push of package to Pypi or Test Pypi

Key steps include:
- Removing old build files to avoid conflicts
- Build package with `build` (install with `pip install build`)
- Upload of package to repository (i.e., Pypi or Test Pypi) with twine
- Creation of new conda environment
- Test install of package by downloading into new environment

In [2]:
import os
import subprocess
import sys
import argparse

### Notes on additional installs

This process assumes anaconda, build, and twine have been installed.

These packages are not imported by this process, but calls are made to them through the command line. 

See [Anaconda install page](https://docs.anaconda.com/anaconda/install/) for install details. Future updates may extend environment testing to venv. 

```python
pip install build
pip install twine
```

In [4]:
def run_command(command, cwd=None):
    """Helper function to run a shell command and handle errors."""
    result = subprocess.run(command, shell=True, cwd=cwd)
    if result.returncode != 0:
        print(f"Command failed: {command}")
        sys.exit(1)

def clean_build(repo_path):
    """Remove build artifacts."""
    print("Cleaning old build artifacts...")
    run_command(f"rm -rf {os.path.join(repo_path, 'dist/')} {os.path.join(repo_path, 'build/')} {os.path.join(repo_path, '*.egg-info')}")

def build_package(repo_path):
    """Build the package."""
    print("Building the package...")
    run_command("python -m build", cwd=repo_path)

def upload_package(repo_path, repository):
    """Upload the package to the specified repository (PyPI or Test PyPI)."""
    print(f"Uploading package to {repository}...")
    if repository == "pypi":
        run_command(f"twine upload {os.path.join(repo_path, 'dist/*')}")
    else:
        run_command(f"twine upload --repository testpypi {os.path.join(repo_path, 'dist/*')}")

def create_conda_env(env_name):
    """Create a clean conda environment."""
    print(f"Creating a clean conda environment: {env_name}")
    run_command(f"conda create -n {env_name} python=3.11 -y")

def install_package(env_name, package_name, repository):
    """Install the package from the specified repository (Test PyPI or PyPI)."""
    print(f"Installing the package '{package_name}' from {repository} in environment '{env_name}'...")
    if repository == "pypi":
        run_command(f"conda activate {env_name} && pip install {package_name}")
    else:
        run_command(f"conda activate {env_name} && pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple {package_name}")


In [5]:
def full_pipeline(repo_path, env_name="testenv", package_name="datascifuncs", repository="testpypi"):
    """Run the full pipeline: clean, build, upload, create env, install."""
    clean_build(repo_path)
    build_package(repo_path)
    upload_package(repo_path, repository)
    create_conda_env(env_name)
    install_package(env_name, package_name, repository)

In [6]:
# Settings
path='/Users/dsl/Documents/GitHub/DataSciFuncs/'
env_name='test_dsf'
package_name='datascifuncs'
repository='testpypi'

In [8]:
# Run the full pipeline with the provided arguments
full_pipeline(path, env_name=env_name, package_name=package_name, repository=repository)

Cleaning old build artifacts...
Building the package...
* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools >= 40.8.0
* Getting build dependencies for sdist...
running egg_info
creating datascifuncs.egg-info
writing datascifuncs.egg-info/PKG-INFO
writing dependency_links to datascifuncs.egg-info/dependency_links.txt
writing requirements to datascifuncs.egg-info/requires.txt
writing top-level names to datascifuncs.egg-info/top_level.txt
writing manifest file 'datascifuncs.egg-info/SOURCES.txt'
reading manifest file 'datascifuncs.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'datascifuncs.egg-info/SOURCES.txt'
* Building sdist...
running sdist
running egg_info
writing datascifuncs.egg-info/PKG-INFO
writing dependency_links to datascifuncs.egg-info/dependency_links.txt
writing requirements to datascifuncs.egg-info/requires.txt
writing top-level names to datascifuncs.egg-info/top_level.txt
reading manifest f

/opt/anaconda3/lib/python3.11/getpass.py:91: GetPassWarning: Can not control echo on the terminal.
  passwd = fallback_getpass(prompt, stream)
Enter your API token: Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/getpass.py", line 69, in unix_getpass
    old = termios.tcgetattr(fd)     # a copy to save
          ^^^^^^^^^^^^^^^^^^^^^
termios.error: (25, 'Inappropriate ioctl for device')

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/bin/twine", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/twine/__main__.py", line 33, in main
    error = cli.dispatch(sys.argv[1:])
            ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/twine/cli.py", line 123, in dispatch
    return main(args.args)
           ^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/twine/commands/upload.

SystemExit: 1

/opt/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
